

## Chapter 5: Mutating Data - Write Operations

In the previous chapter, we mastered the art of fetching data with Queries. However, most applications are not just read-only; they need to create, update, and delete data. In GraphQL, these operations are called **Mutations**.

While a Query is used to read data, a Mutation is used to modify server-side data. Conceptually, they are similar—both have fields, arguments, and return types—but the execution semantics differ significantly.

### 5.1 Anatomy of a GraphQL Mutation

Syntactically, a mutation looks almost identical to a query. The key differences are the use of the `mutation` keyword and the naming convention (typically verbs).

**Basic Example:**

```graphql
mutation CreateNewUser {
  createUser(name: "Alice", email: "alice@example.com") {
    id
    name
    email
  }
}
```

**Breakdown:**
1.  **`mutation`**: The operation type.
2.  **`createUser`**: The mutation field. This triggers the resolver that performs the write operation on the server.
3.  **Arguments**: `name` and `email` are passed to the resolver.
4.  **Selection Set**: Crucially, mutations return an object. Here, we ask for the `id`, `name`, and `email` of the newly created user.

**Why return the object?**
In REST, a `POST` request might return `201 Created` with no body, or just the ID. In GraphQL, it is standard practice to return the modified object (or at least the changed fields). This allows the client to update its local cache immediately without making a follow-up query to fetch the new data.

### 5.2 Designing Mutation Payloads

Designing mutations requires careful thought. Industry guidelines strongly recommend following specific patterns to ensure the API is scalable and easy to use.

#### 5.2.1 The "Input" Pattern: Single Object Arguments

When creating an object, you might have many fields. Passing them as individual arguments becomes messy and hard to maintain.

**Bad Practice (Individual Arguments):**
```graphql
type Mutation {
  createProduct(
    name: String!, 
    price: Float!, 
    description: String, 
    category: ProductCategory!, 
    tags: [String!]!
  ): Product
}
```
This mutation signature is long and brittle. If you want to add a `weight` field, you have to update the schema signature and every client call.

**Best Practice (Input Type):**
Use a single argument of an **Input Type**. This mirrors the structure of the object you are creating and makes the signature clean.

```graphql
input CreateProductInput {
  name: String!
  price: Float!
  description: String
  category: ProductCategory!
  tags: [String!]!
}

type Mutation {
  createProduct(input: CreateProductInput!): ProductPayload!
}
```

#### 5.2.2 Structuring Successful Responses

A mutation shouldn't just return the object; it should return a **Payload Object**. This allows you to return metadata alongside the mutation result.

**Scenario:** When creating a product, you might want to return the product *and* the total count of products now available.

```graphql
type ProductPayload {
  product: Product
  success: Boolean!
  message: String
}

type Mutation {
  createProduct(input: CreateProductInput!): ProductPayload!
}
```

**Query Example:**
```graphql
mutation AddProduct {
  createProduct(input: { name: "Laptop", price: 999.99, category: ELECTRONICS }) {
    product {
      id
      name
    }
    success
    message
  }
}
```

### 5.3 Handling Mutation Errors at the Schema Level

In REST, errors are often communicated via HTTP status codes (4xx, 5xx). In GraphQL, the HTTP status is usually always 200 OK, even if there is a logic error.

**The Two Types of Errors:**
1.  **Global Errors:** Network failures or syntax errors (handled by the `errors` array in the JSON response).
2.  **Domain Errors:** Logic failures like "Email already exists" or "Product out of stock".

**Industry Guideline:** Do not use the top-level `errors` array for domain errors. Instead, model errors in your schema. This makes them type-safe and easier for clients to handle.

**Design Pattern: The Union or Interface for Errors**

A robust pattern is to return a Union type that can be either a Success or an Error.

```graphql
# 1. Define the Success Payload
type CreateUserPayload {
  user: User!
}

# 2. Define the Error Payload
type UserInputError {
  message: String!
  field: String # Which field caused the error?
}

# 3. Create a Union
union CreateUserResult = CreateUserPayload | UserInputError

# 4. Use in Mutation
type Mutation {
  createUser(input: CreateUserInput!): CreateUserResult!
}
```

**Client Handling:**
The client must now handle the result type explicitly using inline fragments.

```graphql
mutation Register {
  createUser(input: { email: "test@test.com", password: "123" }) {
    ... on CreateUserPayload {
      user {
        id
        email
      }
    }
    ... on UserInputError {
      message
      field
    }
  }
}
```

This forces the client developer to think about error cases during development, leading to more robust UIs.

### 5.4 Nested Mutations: Pros, Cons, and Alternatives

In GraphQL, mutations are fields on the `Mutation` type. Technically, you *could* design mutations on object types themselves (e.g., `user.updateEmail(...)`), allowing for nested mutations.

**Why this is generally avoided:**
The GraphQL specification defines that top-level mutation fields are executed **sequentially** (one after another), while query fields are executed in parallel. This ensures data consistency.

If you nest mutations:
1.  It breaks the sequential guarantee.
2.  It makes tracking side effects difficult for the server.
3.  It complicates caching strategies.

**Best Practice:** Keep all "write" operations as top-level fields in the `Mutation` type. Use arguments to target specific IDs.

**Correct Approach:**
```graphql
type Mutation {
  updateUserEmail(userId: ID!, newEmail: String!): User!
}
```

### 5.5 Optimistic UI Updates (Conceptual Overview)

One of the greatest benefits of GraphQL mutations on the client side is **Optimistic UI**.

Since the mutation returns the updated data immediately, the client can assume the mutation will succeed and update the UI *before* the server response arrives.

**The Flow:**
1.  **User Action:** User clicks "Like" on a post.
2.  **Optimistic Update:** Client immediately updates the UI to show the "Liked" state (locally).
3.  **Mutation Sent:** Client sends `mutation LikePost { ... }`.
4.  **Server Response:**
    *   **Success:** UI remains as is (already updated).
    *   **Failure:** Client rolls back the UI to the previous state.

This creates a lightning-fast user experience, making the network latency invisible to the user. Libraries like Apollo Client handle this "rollback" logic automatically if you define the optimistic response.

---

### 🚀 Next Up: Chapter 6 - Real-Time Data - Subscriptions

**Summary:** We can read data and write data. The final piece of the CRUD puzzle is listening to data changes. In Chapter 6, we will introduce Subscriptions. We will learn how to set up real-time data streams, the role of WebSockets, and how to filter subscription events so clients only receive the updates they care about.